In [ ]:
import pandas as pd
import numpy as np
import fiona
import shapely
import folium
import os
import json

from shapely.geometry import Point
from geopandas import GeoDataFrame, read_file
from geopandas.tools import overlay, sjoin

In [ ]:
# Brazil States
root = os.getcwd()
brazil_shp = read_file(root+'/BRA_adm_shp/BRA_adm1.shp')

# Generate some points
#b = [int(x) for x in polydf.total_bounds]
#N = 8
#pointdf = GeoDataFrame([
#    {'geometry' : Point(x, y), 'value1': x + y, 'value2': x - y}
#    for x, y in zip(range(b[0], b[2], int((b[2]-b[0])/N)),
#                    range(b[1], b[3], int((b[3]-b[1])/N)))])

# Upload Brazil Gdelt points

brazil_shp

In [ ]:
gdelt_stories = pd.read_csv(root+"/2016Banks.csv", header=None, names=["lat", "lon"])
# Remember to make points as lon, lat
gdelt_stories['geometry'] = gdelt_stories.apply(lambda row: Point(row.lon, row.lat), axis=1)
gdelt_stories = GeoDataFrame(gdelt_stories).drop(["lon", "lat"], axis=1)
gdelt_stories["tags"] = ["todo" for i in range(gdelt_stories.shape[0])]
gdelt_stories

In [ ]:
join_left_df = sjoin(brazil_shp, gdelt_stories, how="left")
# https://github.com/geopandas/geopandas/issues/325
# This will fail with an "empty to concatenate" message if 
join_left_df

In [ ]:
join_left_df.shape

In [ ]:
# Sum up the number of articles by brazil feature
# Use the geopandas dataframe to make the geo_json

# Use groupby?
# shortcut, need to do significant work to get this to count by tags when that info is here
region_counts = brazil_with_gdelt_stories.groupby(["ID_1"]).count()["tags"]
# Set region_counts to 0 because the center of the country
# gets so many hits from the stories tagged just "Brazil"
# Can work around this by filtering out the points from the CSV above that correspond to only Brazil
## Because this way, we don't see any articles that were coded at a detailed level that happened
## To also fall in the shape shared by the center of the country
region_counts[12]=0
region_counts


In [ ]:
root = os.getcwd() + "/"
GADM_Brazil_dest = root + "BRA_adm_shp/BRA_adm1.shp"

Brazil_features = []
ids = []
with fiona.collection(GADM_Brazil_dest, "r") as source:
    for feature in source:
        # Update properties to include GDELT data
        idx = int(feature["id"])
        feature["id"] = idx
        ids.append(idx)
        Brazil_features.append(feature)
    #crs = " ".join("+%s=%s" % (k,v) for k,v in source.crs.items())

my_layer = {
    "type": "FeatureCollection",
    "features": Brazil_features,
    "crs": {
        "type": "link", 
        "properties": {"href": "my_layer.crs", "type": "proj4"} }}

with open("Brazil_GADM.json", "w") as f:
    f.write(json.dumps(my_layer))

    
#print(Brazil_polys)

In [ ]:
ids = sorted(ids)
rng = list(range(0,len(ids)))
print(rng == ids)
print(len(ids))

In [ ]:
data_df = pd.DataFrame({"id":ids,
                       "num_stories":region_counts})

In [ ]:
data_df.head()

In [ ]:
geo_json = r'Brazil_GADM.json'

map_osm = folium.Map(location=[-22.9068, -40.1729], zoom_start=3)

map_osm.choropleth(geo_path=geo_json, data = data_df,
                   columns=['id', 'num_stories'],
                   key_on='feature.id',
                   fill_color='YlOrRd', 
                 fill_opacity=0.7, line_opacity=0.3,
                legend_name="Number of Stories")

## Notice that most stories are around Rio
# Filtering for different tags you may see interesting stories

map_osm

